<center><h1>POD-Mini: minimized proof of delivery</h1></center>
<center><b>Ryan Kung<br> ryankung(at)ieee.org</b></center>

## Intro

This is a Python implementaton of Pod-Mini of [zkPod](https://github.com/sec-bit/zkPoD-node) via klefki, for more details, just check the [technial Paper](https://github.com/sec-bit/zkPoD-node).

## Preliminaries

In [28]:
from klefki.types.algebra.concrete import EllipticCurveCyclicSubgroupSecp256k1 as Cruve
from klefki.types.algebra.concrete import EllipticCurveGroupSecp256k1 as ECG
from klefki.types.algebra.concrete import FiniteFieldSecp256k1 as F

### Pedersen commitment

$$
Comm(a;b) = g^a \cdot h^b
$$

In [2]:
import random

N = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFF
random_f = lambda: F(random.randint(1, N) % F.P)

q = random_f()
G = Cruve.G
H = G @ q

com = lambda a, b: G@a + H@b
    

### Pedersen commitment for Vector

We generate vector of $g$ point, $\mathbf{G}$ via $[g^h\leftarrow hash(sec)]$.

In [3]:
from functools import reduce
from hashlib import sha256

secret = "some sec"
Gs = list(map(lambda x: G@x,
              (map(lambda x: int(x, 16), 
                  (map(lambda x: sha256(x.encode()).hexdigest(), sha256(secret.encode()).hexdigest()))))))


In [4]:
vec_mul = lambda a: reduce(lambda x,y: x+y,
                  list(map(lambda a: a[0] @ a[1], zip(Gs, a))))

In [5]:
vec_com = lambda a, b: vec_mul(a) + H @ b

In [6]:
hash = lambda x, y: F(int(sha256(str(x.value + y).encode()).hexdigest(), 16) % N)

## Initializer Phase

In [7]:
data = 123456789
tag = 6

In [8]:
m = F(data)
o = F(tag)

In [9]:
sigma = G @ m + H @ o

## Deliver Phase

In [10]:
k_w = random_f()

k = hash(k_w, 1)
k_ = hash(k_w, 2)
k0 = hash(k_w, 3)
k0_ = hash(k_w, 4)

In [11]:
K = G @ k + H @ k_
K0 = G @ k0 + H @ k0_
(K, K0)

(EllipticCurveGroupSecp256k1::(FiniteFieldSecp256k1::104989255908567702886328443883371995050681185680813196335531669235402508573985, FiniteFieldSecp256k1::12286207758926525150663525523877202066913287190226804579866727773915235253246),
 EllipticCurveGroupSecp256k1::(FiniteFieldSecp256k1::15761946530472198134714043893998952540476268345801161780185411836950408050906, FiniteFieldSecp256k1::115452774037230558222630243755609942345731321646938523138707083966045717569699))

* $S \rightarrow R$: $(K, K_0)$

In [12]:
c = random_f()

* $R \rightarrow S$: $c$

In [13]:
m_ = k + c * m
o_ = k_ + c * o
z = k0 + c * k
z_ = k0_ + c * k_
(m_, o_, z, z_)

(FiniteFieldSecp256k1::44119218700161389723318829011566736228344,
 FiniteFieldSecp256k1::4804118415788978092037101954776695,
 FiniteFieldSecp256k1::1405135653286567184729031096551514974901472551331788477982024166717,
 FiniteFieldSecp256k1::950565649861140730356224913070793591140531404638331317731018300947)

* $S \rightarrow R: (\bar{m}, \bar{o}, z, z')$

R should verify that:

$$
Com(m; o)^c \cdot Com(k_0;k'_0) \stackrel{?}{=} Com(\bar{m}; \bar{o})\\
Com(k_0;k'_0) \cdot Com(k;k')^c \stackrel{?}{=} Com(z;z')
$$

In [14]:
sigma @ c + K == G @ m_ + H @ o_

True

In [15]:
K0 + (K @ c) == G @ z + H @ z_

True

### Reveal-phase

* $R \rightarrow J: \rho$
* $S \rightarrow J: k_{\omega}$

$$
z \stackrel{?}{=} H(k_{\omega}, 3) + c \cdot H(k_{\omega}, 1)
$$

In [24]:
assert z == hash(k_w, 3) + c @ hash(k_w, 1)

In [26]:
assert m == (m_ - hash(k_w, 1)) / c

In [27]:
(m_ - hash(k_w, 1)) / c

FiniteFieldSecp256k1::123456789